# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from openai import AzureOpenAI
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [2]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('AZURE_OPENAI_API_KEY')
endpoint = os.getenv('ENDPOINT')
version = os.getenv('VERSION')
MODEL = os.getenv('DEPLOYMENT')

client = AzureOpenAI(
    azure_endpoint=endpoint, 
    api_key=api_key,
    api_version=version
)

In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with Selenium for dynamic websites.
    """

    def __init__(self, url):
        self.url = url

        # Set up Selenium WebDriver with Chrome
        options = webdriver.ChromeOptions()
        options.add_argument('--headless')  # Run in headless mode
        options.add_argument('--disable-gpu')
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # Load the webpage
        driver.get(url)
        time.sleep(5)  # Wait for the page to load completely

        # Get the page source and parse it with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        driver.quit()  # Close the browser

        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""

        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [5]:
ed = Website("https://edwarddonner.com")
ed.links

['#wp--skip-link--target',
 'https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwar

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [6]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [7]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#wp--skip-link--target
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwardd

In [10]:
def get_links(url):
    website = Website(url)
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}

In [12]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen3-235B-A22B',
 '/deepseek-ai/DeepSeek-Prover-V2-671B',
 '/nari-labs/Dia-1.6B',
 '/Qwen/Qwen3-30B-A3B',
 '/Qwen/Qwen3-32B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Qwen/Qwen3-Demo',
 '/spaces/nari-labs/Dia-1.6B',
 '/spaces/stepfun-ai/Step1X-Edit',
 '/spaces/RiverZ/ICEdit',
 '/spaces',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets/nvidia/Nemotron-CrossThink',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/rajpurkarlab/ReXGradient-160K',
 '/datasets/OpenGVLab/InternVL-Data',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '/docs/safe

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
API
Solutions
Research
Commitments
Learn
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
At Anthropic, we build AI to serve humanity’s long-term well-being.
While no one can foresee every outcome AI will have on society, we do know that designing powerful technologies requires both bold 

In [16]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Bro

In [36]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [18]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [19]:
get_brochure_user_prompt("anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}]}


"You are looking at a company called: anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nSolutions\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nAt Anthropic, we build AI to serve humanity’s long-term well-being.\nWhile no one can foresee every outcome AI will have on society, we do know that designing powerful technologies requires both bold steps forward and intentional pauses to consider the effects. That’s why we focus on building tools with human benefit at their foundation, l

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'Discuss page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nQwen/Qwen3-235B-A22B\nUpdated\n4 days ago\n•\n28.9k\n•\n674\ndeepseek-ai/DeepSeek-Prover-V2-671B\nUpdated\n5 days ago\n•\n1.75k\n•\n646\nnari-labs/Dia-1.6B\nUpdated\n8 days ago\n•\n109k\n•\n1.82k\nQwen/Qwen3-30B-A3B\nUpdated\n5 days ago\n•\n41.4k\n•\n440\nQwen/Qwen3-32B\nUpdated\n6 days ago\n•\n75.7k\n•\n267\nBrowse 1M+ models\nSpaces\nRunning\n5.92k\n5.92k\nDeepSite\n🐳\nGenerate any applicati

In [21]:
def create_brochure(company_name, url):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/about'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


# Anthropic Brochure

---

## About Anthropic

Welcome to Anthropic, an innovative public benefit corporation based in San Francisco dedicated to building AI technologies that prioritize safety and humanity's long-term well-being. At Anthropic, we believe in making bold advancements in AI while being mindful of the potential societal impacts our technologies could have. We believe the development of powerful AI requires careful consideration and a commitment to responsible practices.

Our flagship AI model, **Claude 3.7 Sonnet**, represents our commitment to creating intelligent applications that serve human needs effectively. Designed with safety and accountability in mind, Claude exemplifies our core philosophy in AI development.

---

## Our Core Values

At Anthropic, we act for the global good. Our daily decisions are guided by values that reflect our commitment to maximizing positive outcomes for humanity in the long run. Key components of our culture include:

- **Intellectual Curiosity**: We encourage the exploration of innovative ideas and approaches.
- **Collaboration**: Our varied team backgrounds—from machine learning and physics to public policy—help us forecast the broad implications of AI technologies.
- **Responsibility**: Shaping the future of AI is a privilege that we approach with care and intentionality.

---

## Customers and Solutions

We develop AI solutions that empower businesses and individuals to harness the full potential of AI responsibly. Our offerings include:

- **API Access**: Build custom applications and experiences leveraging Claude’s capabilities.
- **AI Agents**: Enhance customer interactions with intelligent support solutions.
- **Coding Assistance**: Streamline programming tasks through AI-powered code suggestions.

Whether you are an enterprise looking to integrate AI into your operations or a developer seeking to create innovative applications, Anthropic is here to support your journey.

---

## Careers at Anthropic

Join our mission of making AI safe and beneficial for all. At Anthropic, we value our team and offer a comprehensive range of benefits to support their professional journeys and personal well-being:

- **Health & Wellness**: Comprehensive health, dental, and vision coverage, along with generous mental health support.
- **Flexible Policies**: Flexible paid time off, inclusive parental leave, and relocation support.
- **Competitive Compensation**: Equity packages, retirement plans, and additional stipends to help you maintain a work-life balance.

We're always looking for passionate individuals to join our team. Explore our open roles and become part of an organization that is committed to shaping the future of AI.

---

## Let’s Build a Safer AI Together

At Anthropic, we are on a mission to create AI technologies that benefit humanity while prioritizing safety and responsibility. By putting AI development at the forefront of our values, we hope to redefine what it means to harness the power of artificial intelligence. 

**Interested in collaborating or learning more?** Visit us at [anthropic.ai](http://anthropic.ai) to explore our models and solutions, or check our careers page to find your place within our dedicated team. 

---

© 2025 Anthropic PBC | All rights reserved.

In [23]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

---

## Welcome to Hugging Face

**Hugging Face** is at the forefront of the artificial intelligence community, driving innovation through collaboration and open-source solutions. We believe in building the future together with our users, contributors, and partners.

### Our Mission

We are dedicated to creating a collaborative platform for the machine learning community. Our tools and models enable researchers and developers to create impactful AI applications that enhance the way we live and work.

---

### Key Offerings

- **Explore AI Apps**: Dive into a comprehensive platform with over **1 million models** and **250,000 datasets**. 
- **Collaboration Tools**: Host and collaborate on unlimited public models, datasets, and applications.
- **Enterprise Solutions**: Provide organizations with advanced, enterprise-grade tools for security, access control, and support.

---

### Company Culture

At Hugging Face, we uphold a culture of openness, inclusivity, and continuous learning. We foster an environment where knowledge is shared, and all voices are encouraged. Our team is built of passionate AI enthusiasts dedicated to pushing the boundaries of what’s possible in machine learning.

---

### Customers & Community

We proudly serve over **50,000 organizations**, including tech giants like **Google**, **Amazon**, and **Microsoft**, ensuring they have the tools necessary to build state-of-the-art AI applications. Our community thrives on collaboration, with contributions from individuals and organizations alike.

---

### Career Opportunities

Join us in shaping the future of AI! We are looking for enthusiastic and skilled individuals to join various teams. At Hugging Face, you can:

- Work with cutting-edge technology in a fast-paced environment.
- Participate in an inclusive and innovative culture.
- Contribute to open-source projects that have a global impact.

Explore our job listings and be a part of the AI revolution!

---

### Get Involved

**Explore Our Tools:**
- [Browse Models](https://huggingface.co/models)
- [Explore Datasets](https://huggingface.co/datasets)
- [Join the Community on Discord](https://huggingface.co/community)

---

Hugging Face stands as a leader in the AI community, committed to fostering collaboration and innovation. Our tools empower individuals and organizations to excel in their AI endeavors, and we invite you to join us in building the future!

**Connect with us!**
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)
- [GitHub](https://github.com/huggingface)

--- 

**Hugging Face: Together, We Build the Future of AI!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [34]:
def stream_brochure(company_name, url):
    stream = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        try:
            # Access attributes directly instead of using .get()
            delta_content = chunk.choices[0].delta.content if chunk.choices else ""
            if delta_content:
                response += delta_content
                response = response.replace("```", "").replace("markdown", "")
                update_display(Markdown(response), display_id=display_handle.display_id)
        except (IndexError, AttributeError) as e:
            print(f"⚠️ Error processing chunk: {e}")

    return response

In [35]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Company Brochure

---

## Company Overview

**Anthropic** is a pioneering organization deeply committed to developing AI technologies with a focus on human benefit and safety. Our flagship AI model, **Claude 3.7 Sonnet**, represents the forefront of our commitment to creating responsible, ethical, and aligned artificial intelligence solutions that prioritize the long-term well-being of humanity. We emphasize continuous innovation paired with cautious reflection to anticipate the societal impacts of our technologies.

### Our Mission
At Anthropic, our mission is to ensure that AI serves humanity’s long-term well-being. We achieve this through rigorous research, effective policy work, and designing products that embody our core commitment to responsible AI development.

## Core Values

- **Safety First**: We prioritize AI safety in all aspects of our work, ensuring our technologies enhance rather than inhibit human quality of life.
- **Transparency**: We maintain open channels about our operations, research findings, and scaling policies, promoting trust and credibility with our stakeholders.
- **Collaboration**: We believe in building technologies that foster collaboration, both within our team and with our users.

## Our Products

- **Claude**: A state-of-the-art AI assistant designed to learn and adapt to user needs. Claude powers a variety of applications, offering capabilities in coding, customer support, and more.
- **API Solutions**: Developers can easily integrate Claude’s capabilities into their applications through our robust API, which facilitates the creation of innovative AI-powered solutions.

### Services
- Custom AI applications
- Development support and API integration
- Education and training through Anthropic Academy

## Our Customers

We serve a diverse range of industries seeking to leverage AI for practical applications, notably in fields like customer support, software development, and tailored AI solutions. Customer stories highlight our successful collaborations where Claude has significantly improved operational outcomes.

## Company Culture

At Anthropic, we cultivate a culture of **innovation**, **inclusivity**, and **continuous learning**. Our team consists of forward-thinking pioneers passionate about the transformative potential of AI. We prioritize individual well-being and foster a collaborative environment where every voice is heard.

### Commitment to Careers
Join us in our mission to shape the future of safe AI. We are actively looking to expand our team with dynamic individuals who share our vision for a responsible AI landscape.

#### Open Roles
Explore exciting opportunities within Anthropic’s various teams. Each role involves not only technical skill but a commitment to ethical practices in technology.

## Education and Development

Through the **Anthropic Academy**, we provide resources and training to help our partners and clients build effective AI solutions using Claude. Our commitment to education ensures that everyone can access the tools and understanding necessary to thrive in the AI landscape.

---

*Discover more about how Anthropic is leading the way in AI safety and responsible development. Join us in our journey to empower humanity through innovative technology.* 

[Learn more at Anthropic](https://www.anthropic.com)

'# Anthropic Company Brochure\n\n---\n\n## Company Overview\n\n**Anthropic** is a pioneering organization deeply committed to developing AI technologies with a focus on human benefit and safety. Our flagship AI model, **Claude 3.7 Sonnet**, represents the forefront of our commitment to creating responsible, ethical, and aligned artificial intelligence solutions that prioritize the long-term well-being of humanity. We emphasize continuous innovation paired with cautious reflection to anticipate the societal impacts of our technologies.\n\n### Our Mission\nAt Anthropic, our mission is to ensure that AI serves humanity’s long-term well-being. We achieve this through rigorous research, effective policy work, and designing products that embody our core commitment to responsible AI development.\n\n## Core Values\n\n- **Safety First**: We prioritize AI safety in all aspects of our work, ensuring our technologies enhance rather than inhibit human quality of life.\n- **Transparency**: We mainta

In [37]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}



# Welcome to Hugging Face: The Software Hug We All Need! 🫂

### 🚀 Who Are We?
At **Hugging Face**, we're the friendly neighborhood AI enthusiasts! Our mission? To build the future with cutting-edge models, datasets, and all things machine learning—like superheroes, but with laptops instead of capes. 

### 🧑‍🤝‍🧑 Company Culture: A Hug for Everyone
Join a community where collaboration feels less like a corporate job and more like a friendly gathering of nerdy friends. Coffee breaks? Always in the AI lab!  
- **Vibe Check:** Casual yet productive. Like lounging around in pajamas, sipping coffee, and discussing neural networks at the same time.
- **Fun Perks:** Regular "model" reviews where everyone can softly critique each other’s creations over donuts (AI makes them low-cal!).

### 🌏 Our Customers: Everyone Who Wants a Hug!
From non-profits to tech giants like Google and Microsoft, we cater to over **50,000 organizations**! If you want to scream *"AI hug me!"*, we’ve got just the thing for you:
- **Enterprise-grade solutions**, for those who like their hugs with a side of security and dedicated support.
- A treasure trove of **1M+ models** where you can discover everything from quirky chatbots to serious data wizards.

### 💼 Career Opportunities: Hugonomics! 
Looking to jump into the world of machine learning? We have jobs! And no, we don’t need you to hug the competition—just have a knack for creativity and a love for learning.
- **Positions available:** From AI groundbreakers to community coordinators. Bring your sense of humor and passion for tech!
  
### 🌟 What We Offer:
- A **collaboration platform** that makes building AI as easy as pie (and just as delicious).
- Access to **cutting-edge tools and datasets** while you sip your morning latte. 
- Supportive atmosphere where one day, you're a model creator, and the next, you’re making AI say “I love you” in 10 different languages.

### 🤖 Explore Our Universe:
- **Models**: Dive into the bazillion(ish) models at your fingertips.
- **Datasets**: Access to 250k+ datasets (who knew data could be this fun?).
- **Spaces**: Create cool applications to showcase your brilliance to the world!

### 💬 Get in Touch
Don’t be shy—join our community on our social channels! We talk about everything from the latest AI trends to whether pineapple belongs on pizza.  
- **Twitter:** @HuggingFace  
- **Discord:** Your new hangout spot!  

**Hugging Face**: Where AI lovers unite for a better future, one code snippet at a time! So come on in, the virtual hugs are waiting. 🫂✨



'\n# Welcome to Hugging Face: The Software Hug We All Need! 🫂\n\n### 🚀 Who Are We?\nAt **Hugging Face**, we\'re the friendly neighborhood AI enthusiasts! Our mission? To build the future with cutting-edge models, datasets, and all things machine learning—like superheroes, but with laptops instead of capes. \n\n### 🧑\u200d🤝\u200d🧑 Company Culture: A Hug for Everyone\nJoin a community where collaboration feels less like a corporate job and more like a friendly gathering of nerdy friends. Coffee breaks? Always in the AI lab!  \n- **Vibe Check:** Casual yet productive. Like lounging around in pajamas, sipping coffee, and discussing neural networks at the same time.\n- **Fun Perks:** Regular "model" reviews where everyone can softly critique each other’s creations over donuts (AI makes them low-cal!).\n\n### 🌏 Our Customers: Everyone Who Wants a Hug!\nFrom non-profits to tech giants like Google and Microsoft, we cater to over **50,000 organizations**! If you want to scream *"AI hug me!"*, w

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>